In [54]:
from random import randint
from functools import cmp_to_key
import math
from skimage import color

florestColor = [100,35,50] #cor da florsta, que o inseto deve se adaptar em LAB
maxPopulation = 1000 #população maxima
floorMutationNumber = 96 # numero randomico que o algoritmo de mutação se baseia, para realizar, ou não, a mutação
mutationIncrease = 5 # qtd de cor que a mutação adiciona ou decrementa
goodsCreaturePerPopulation = 3 # ELITISMO -  qtd de melhores individuos mantidos por cada nova geração

rF = florestColor[0] 
gF = florestColor[1]
bF = florestColor[2]

In [55]:

def crossover(bug1,bug2): # gera um novo individuo, com base em outro
    bugOneColors = [bug1.r, bug1.g,bug1.b]
    bugTwoColors = [bug2.r, bug2.g,bug2.b]
    randomColor = randint(0, 2)
    bugTwoColors[randomColor] = bugOneColors[randomColor]
    newBug = Bug()
    newBug.r = bugTwoColors[0]
    newBug.g = bugTwoColors[1]
    newBug.b = bugTwoColors[2]
    newBug.setFitness()
    return newBug

def mutation(bug1): #realiza uma mutação, ou não, com base em um valor randomico

  randomNumber = randint(0,100)

  if randomNumber >= floorMutationNumber:
    bugColor = [bug1.r, bug1.g, bug1.b]
    randomColor = randint(0,2)
    randomOperation = randint(0,100)
    if randomOperation <= 50:
      bugColor[randomColor] = bugColor[randomColor] + mutationIncrease;
    else:
      bugColor[randomColor] = bugColor[randomColor] - mutationIncrease;

    if bugColor[randomColor] > 255:
        bugColor[randomColor] = bugColor[randomColor] - (mutationIncrease * 3)
    if bugColor[randomColor] < 0:
        bugColor[randomColor] = bugColor[randomColor] * -1 

    bug1.r = bugColor[0]
    bug1.g = bugColor[1]
    bug1.b = bugColor[2]
    bug1.setFitness()
    return bug1;
  else:
    return bug1;

def fitnessAverage(bugs):
    average = 0.0
    for x in bugs:
      average += x.fitness

    return float(average/len(bugs))




In [56]:
class Bug:
    
    def __init__(self):
        self.r = randint(0,255)
        self.g = randint(0,255)
        self.b = randint(0,255)
        self.fitness = float( (((abs(rF - self.r)/255) + (abs(gF - self.g)/255) + (abs(bF - self.b)/255) )/3) * 100) 

    def __str__(self):
        return '[{},{},{}]'.format(self.r, self.g, self.b)

    def setFitness(self):
       self.fitness = float( (((abs(rF - self.r)/255) + (abs(gF - self.g)/255) + (abs(bF - self.b)/255) )/3) * 100) 


In [57]:
def bug_comparator(Bug1, Bug2):
   if Bug1.fitness < Bug2.fitness:
      return -1
   elif Bug1.fitness > Bug2.fitness:
      return 1
   else:
      return 0

In [58]:
class Population:

  def __init__(self):
    self.bugs = []

  def createPopulation(self,maxPopulation):
    for i in range(maxPopulation):
      self.bugs.append(Bug())

    return self.bugs;

  def getMaxFitnessBugsCurrentPopulation(self): #retorna os melhores individuos da população
    ordered = sorted(self.bugs, key = cmp_to_key(bug_comparator), reverse=True)
    return ordered[-10:] #retorna os N ultimos elementos da lista ordenada (os melhores fitness)

  def createNewBug(self):
    crossOrMutation = randint(0,100)  
    if crossOrMutation <= 50:
      return crossover(self.bugs[randint(0,len(self.bugs) - 1)], self.bugs[randint(1,len(self.bugs)-1)])
    else:
      return mutation(self.bugs[randint(0,len(self.bugs) - 1)])

  

  

In [66]:
fitnessFloor = 0.03;

def generateNewPopulations():
    initialPopulation = Population()
    initialPopulation.createPopulation(maxPopulation) #cria população com o numero MaxPopulation de individuos
    #porcentagem de fitness dos melhores individuos da população
    maxGenerations = 1000
    currentGeneration = 1;


    currentPopulation = initialPopulation;
    averageFit = fitnessAverage(currentPopulation.getMaxFitnessBugsCurrentPopulation())
    #condição de parada se baseia em atingir um fitnessFloor ou N max de gerações | poderiamos tbm verificar as ultimas 3 gerações
    # e se o fitness global continuasse igual ou mt proximo, parar o algoritmo
    while averageFit >= fitnessFloor and currentGeneration < maxGenerations:
      
      newBugsPopulation = currentPopulation.getMaxFitnessBugsCurrentPopulation() #elitismo -> recuperando melhores insetos da população atual
      while (len(newBugsPopulation) < maxPopulation): #enquanto a população nova n tiver o mesmo tamanho da antiga continua enchendo o array
          newBugsPopulation.append(currentPopulation.createNewBug()) #gera novos individuos com base na população atual


      currentPopulation.bugs = newBugsPopulation
      averageFit = fitnessAverage(currentPopulation.getMaxFitnessBugsCurrentPopulation())
      currentGeneration += 1;

    return currentPopulation;


populationFinal = generateNewPopulations()

In [67]:
print("Florest Color: ", florestColor)
print("Fitness minimum:", fitnessFloor)
print("Best Bugs in Population color:")
for x in populationFinal.getMaxFitnessBugsCurrentPopulation():
  print(x)


Florest Color:  [100, 35, 50]
Fitness minimum: 2.03
Best Bugs in Population color:
[89,35,54]
[89,35,54]
[89,35,54]
[89,35,54]
[89,35,54]
[89,35,54]
[89,35,54]
[89,35,54]
[89,35,54]
[99,35,54]
